# Machine Learning Capstone Project - Starbucks


Importing necessary libraries and datasets

In [4]:
import pandas as pd
import numpy as np
import math
import json


# read in the json files
portfolio = pd.read_json('portfolio.json', orient='records', lines=True)
profile = pd.read_json('profile.json', orient='records', lines=True)
transcript = pd.read_json('transcript.json', orient='records', lines=True)

In [6]:
portfolio.head()

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7


In [7]:
profile.head()

,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


In [8]:
transcript.head()

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


In [16]:
transcript['offers']=transcript['event'].apply(lambda x:1 if x=='offer completed' else 0 if x=='offer received' else -1)

In [18]:
transcript.offers.value_counts()

-1    196678
 0     76277
 1     33579
Name: offers, dtype: int64

In [21]:
transcript.event.value_counts()


transaction        138953
offer received      76277
offer viewed        57725
offer completed     33579
Name: event, dtype: int64

In [22]:
transcript[(transcript['event']=='offer completed')&(transcript['value']=='offer completed')]


,person,event,value,time,Successful_offer,offers
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0,0,0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0,0,0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0,0,0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0,0,0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0,0,0
...,...,...,...,...,...,...
257882,d087c473b4d247ccb0abfef59ba12b0e,offer received,{'offer id': 'ae264e3637204a6fb9bb56bc8210ddfd'},576,0,0
257883,cb23b66c56f64b109d673d5e56574529,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},576,0,0
257884,6d5f3a774f3d4714ab0c092238f3a1d7,offer received,{'offer id': '2298d6c36e964ae4a3e7e9706d1fb8c2'},576,0,0
257885,9dc1421481194dcd9400aec7c9ae6366,offer received,{'offer id': 'ae264e3637204a6fb9bb56bc8210ddfd'},576,0,0


In [26]:
offers_received=transcript[(transcript['event']=='offer completed')]


In [32]:
offers_received['offer_ids']=offers_received['value'].apply(lambda x:x['offer_id'])

C:\Users\diegbuca\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [34]:
offers_received[offers_received['offer_ids']=='9b98b8c7a33c4b65b9aebfe6a799e6d9']

,person,event,value,time,Successful_offer,offers,offer_ids
12679,629fc02d56414d91bca360decdfa9288,offer completed,{'offer_id': '9b98b8c7a33c4b65b9aebfe6a799e6d9...,0,1,1,9b98b8c7a33c4b65b9aebfe6a799e6d9
12780,b860d355ef6e4c66b5d5a837c56ef32d,offer completed,{'offer_id': '9b98b8c7a33c4b65b9aebfe6a799e6d9...,0,1,1,9b98b8c7a33c4b65b9aebfe6a799e6d9
12784,99297ea01107436fa8c2e2bc86f55d89,offer completed,{'offer_id': '9b98b8c7a33c4b65b9aebfe6a799e6d9...,0,1,1,9b98b8c7a33c4b65b9aebfe6a799e6d9
12936,16dc4306258b4fef90ffe99f53ed7952,offer completed,{'offer_id': '9b98b8c7a33c4b65b9aebfe6a799e6d9...,0,1,1,9b98b8c7a33c4b65b9aebfe6a799e6d9
13021,0454292436bd4cee98e8ec873e30433e,offer completed,{'offer_id': '9b98b8c7a33c4b65b9aebfe6a799e6d9...,0,1,1,9b98b8c7a33c4b65b9aebfe6a799e6d9
...,...,...,...,...,...,...,...
306348,087d0fcdbf294f7494a96d8d6a56cf41,offer completed,{'offer_id': '9b98b8c7a33c4b65b9aebfe6a799e6d9...,714,1,1,9b98b8c7a33c4b65b9aebfe6a799e6d9
306410,b13eaeb7a91e415a972c9ebc7f63ac57,offer completed,{'offer_id': '9b98b8c7a33c4b65b9aebfe6a799e6d9...,714,1,1,9b98b8c7a33c4b65b9aebfe6a799e6d9
306423,8771f96bc7f947c09a8f78ee2e14a1a7,offer completed,{'offer_id': '9b98b8c7a33c4b65b9aebfe6a799e6d9...,714,1,1,9b98b8c7a33c4b65b9aebfe6a799e6d9
306430,0355c6a5fdbc429ea5b05e8a9ecd2eae,offer completed,{'offer_id': '9b98b8c7a33c4b65b9aebfe6a799e6d9...,714,1,1,9b98b8c7a33c4b65b9aebfe6a799e6d9
